In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re

In [2]:
%%time
nrows=1110
df_train = pd.read_table('train_data.txt',nrows=nrows)
df_train['id'], df_train['reactants>reagents>production'] = df_train['id,reactants>reagents>production'].str.split(',', 1).str
del df_train['id,reactants>reagents>production']
df_train['reactants'],df_train['reagents'],df_train['production'] = df_train['reactants>reagents>production'].str.split('>').str
del df_train['reactants>reagents>production']
df_train.index = df_train['id']
del df_train['id']

CPU times: user 46.5 ms, sys: 6.98 ms, total: 53.5 ms
Wall time: 59.3 ms


In [3]:
def get(i):
    try:
        return i.replace(i[i.index('|'):],'').strip()
    except:
        return i.strip()
df_train['production'] = df_train['production'].apply(lambda x:get(x))

In [4]:
df = pd.concat([df_train['reactants'],df_train['reagents'],df_train['production']])
documents = df.tolist()
documents_after = [' '.join(i.split('.')) for i in documents]
print(len(documents_after))
documents_after[0] +'  '

3330


'[ClH:51] C([O-])(=[O:46])C C([OH:50])C [CH3:1][CH2:2][C:3]1[C:12]2[CH2:13][N:14]3[C:19](=[O:20])[C:18]4[CH2:21][O:22][C:23]([C@:25]([OH:28])([CH2:26][CH3:27])[C:17]=4[CH:16]=[C:15]3[C:11]=2[N:10]=[C:9]2[C:4]=1[CH:5]=[C:6]([O:29][C:30]([N:32]1[CH2:37][CH2:36][CH:35]([N:38]3[CH2:43][CH2:42][CH2:41][CH2:40][CH2:39]3)[CH2:34][CH2:33]1)=[O:31])[CH:7]=[CH:8]2)=[O:24]  '

In [5]:
import keras
from keras.preprocessing import text as T
vectorizer = T.Tokenizer(lower=False,split=
' ',num_words=None,char_level=False,filters='')

Using TensorFlow backend.


In [6]:
vectorizer.fit_on_texts(documents_after)
text_sequences = vectorizer.texts_to_sequences(documents_after)

In [7]:
index_word_df = pd.DataFrame.from_dict(vectorizer.index_word, 
                                       orient='index')
print('总计单词数：%s' % index_word_df.shape[0])
print('最大建模用索引下标:%s' % np.max([np.max(t) for t in text_sequences if len(t)>0]), '\n')
word_counts_df = pd.DataFrame.from_dict(vectorizer.word_counts, 
                                        orient='index', 
                                        columns=['counts'])
word_index_df = pd.DataFrame.from_dict(vectorizer.word_index, 
                                       orient='index', 
                                       columns=['index'])
temp=pd.merge(word_counts_df, word_index_df, 
              left_index=True, right_index=True)
print(temp.sort_values(by='counts', ascending=False).head(10))
print('最大单词长度：',np.max([len(x) for x in text_sequences] ))

总计单词数：3575
最大建模用索引下标:3575 

         counts  index
O           285      1
[Na+]       267      2
Cl          167      3
CO          140      4
[K+]        134      5
[OH-]       110      6
C1COCC1     107      7
C(Cl)Cl      98      8
O1CCCC1      97      9
ClCCl        87     10
最大单词长度： 10


In [8]:
index_word = vectorizer.index_word
index_word.update({0:' '})
word_index = vectorizer.word_index
word_index.update({' ':0})

In [9]:
index_word

{1: 'O',
 2: '[Na+]',
 3: 'Cl',
 4: 'CO',
 5: '[K+]',
 6: '[OH-]',
 7: 'C1COCC1',
 8: 'C(Cl)Cl',
 9: 'O1CCCC1',
 10: 'ClCCl',
 11: 'CN(C=O)C',
 12: 'C(O)C',
 13: 'C(=O)([O-])[O-]',
 14: '[Pd]',
 15: 'C(N(CC)CC)C',
 16: 'C(OCC)(=O)C',
 17: 'C1(C)C=CC=CC=1',
 18: 'CN(C)C=O',
 19: '[Cl-]',
 20: '[Li+]',
 21: '[H-]',
 22: 'O1CCOCC1',
 23: 'C([O-])(=O)C',
 24: 'C([O-])([O-])=O',
 25: 'N1C=CC=CC=1',
 26: 'C(O)(=O)C',
 27: 'C(#N)C',
 28: 'C1(P(C2C=CC=CC=2)C2C=CC=CC=2)C=CC=CC=1',
 29: 'C1C=CC(/C=C/C(/C=C/C2C=CC=CC=2)=O)=CC=1',
 30: 'CCOC(C)=O',
 31: 'CS(C)=O',
 32: 'C(OCC)C',
 33: 'C1C=CC(P(C2C=CC=CC=2)[C-]2C=CC=C2)=CC=1',
 34: 'CCO',
 35: '[NH4+]',
 36: 'C(Cl)(Cl)Cl',
 37: 'C(=O)(O)[O-]',
 38: 'CCN(C(C)C)C(C)C',
 39: '[Cs+]',
 40: 'CCOCC',
 41: '[Al+3]',
 42: 'FC(F)(F)C(O)=O',
 43: '[H][H]',
 44: 'CCCCCC',
 45: 'CC(C)=O',
 46: 'C([O-])(O)=O',
 47: 'C1C=CC([P]([Pd]([P](C2C=CC=CC=2)(C2C=CC=CC=2)C2C=CC=CC=2)([P](C2C=CC=CC=2)(C2C=CC=CC=2)C2C=CC=CC=2)[P](C2C=CC=CC=2)(C2C=CC=CC=2)C2C=CC=CC=2)(C2C=C

In [10]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self,word_index,index_word):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.char_indices = word_index
        self.indices_char = index_word

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.char_indices)))
        for i, c in enumerate(C):
            x[i, c-1] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        print(x)
        return ' '.join(self.indices_char[x] for x in x)
ctable = CharacterTable(word_index,index_word)

In [11]:
from keras.preprocessing import sequence
'''数据预处理'''
def get_input(x):
    reactants = x[0]
    reagents = x[1]
    production = x[2]
    input_seq1 = production.split('.')
    input_seq2 = reagents.split('.')
    input_seq1.extend(input_seq2)
    return input_seq1
input_seq = df_train.apply(lambda x :get_input(x),axis=1)
input_seq = input_seq.tolist()
output_seq = df_train['reactants'].str.split('.').tolist()
input_sequences = vectorizer.texts_to_sequences(input_seq)
output_sequences = vectorizer.texts_to_sequences(output_seq)
np.max([len(x) for x in output_seq])

9

In [12]:
#输入序列的padded
#输出序列的padded
max_len1 = 15
input_sequences_padded = sequence.pad_sequences(input_sequences, maxlen=max_len1)
output_sequences_padded = sequence.pad_sequences(output_sequences, maxlen=max_len1)

In [13]:
input_sequences_padded

array([[   0,    0,    0, ..., 2439, 2440,   16],
       [   0,    0,    0, ...,    0, 2441,   18],
       [   0,    0,    0, ...,   18,    1,   16],
       ...,
       [   0,    0,    0, ..., 3573,    8,    1],
       [   0,    0,    0, ..., 3574,    4,   14],
       [   0,    0,    0, ...,    0, 3575,   36]], dtype=int32)

In [14]:
output_sequences_padded

array([[   0,    0,    0, ...,  272,  273,  274],
       [   0,    0,    0, ...,  276,   13,    5],
       [   0,    0,    0, ...,   88,  277,  278],
       ...,
       [   0,    0,    0, ..., 2374,   31,   15],
       [   0,    0,    0, ...,    0,    0, 2375],
       [   0,    0,    0, ..., 2376, 2377, 2378]], dtype=int32)

In [15]:
#处理成seq2seq需要的数据样本
inputs = np.zeros((nrows, 15, len(word_index)), dtype=np.float32)
for i,C in enumerate(input_sequences_padded):
    inputs[i] = ctable.encode(C,15)
outputs = np.zeros((nrows, 15, len(word_index)), dtype=np.float32)
for i,C in enumerate(input_sequences_padded):
    outputs[i] = ctable.encode(C,15)
X = inputs

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import json

In [23]:
print('Build model...')
from keras.models import Sequential
from keras import layers
RNN = layers.LSTM
HIDDEN_SIZE = 15
BATCH_SIZE = 128
LAYERS = 1
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(X.shape[1],X.shape[2]), return_sequences=True))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#model.add(layers.RepeatVector(15))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True,activation='softmax'))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(word_index), activation='softmax')))
#model.add(layers.Dense(len(word_index), activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 15, 15)            215520    
_________________________________________________________________
lstm_6 (LSTM)                (None, 15, 15)            1860      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 3576)          57216     
Total params: 274,596
Trainable params: 274,596
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(inputs,outputs, batch_size=BATCH_SIZE,epochs=20)

Epoch 1/20
1110/1110 [==============================] - 2s 2ms/step - loss: 8.1240 - acc: 0.0036
Epoch 2/20
1110/1110 [==============================] - 2s 2ms/step - loss: 8.1003 - acc: 0.0036
Epoch 3/20
1110/1110 [==============================] - 2s 2ms/step - loss: 8.0760 - acc: 0.0036
Epoch 4/20
1110/1110 [==============================] - 2s 2ms/step - loss: 8.0504 - acc: 0.0036
Epoch 5/20
1110/1110 [==============================] - 2s 2ms/step - loss: 8.0224 - acc: 0.1926
Epoch 6/20
1110/1110 [==============================] - 2s 2ms/step - loss: 7.9912 - acc: 0.7981
Epoch 7/20
1110/1110 [==============================] - 2s 2ms/step - loss: 7.9561 - acc: 0.8135
Epoch 8/20
1110/1110 [==============================] - 2s 2ms/step - loss: 7.9163 - acc: 0.8135
Epoch 9/20
1110/1110 [==============================] - 2s 2ms/step - loss: 7.8695 - acc: 0.8135
Epoch 10/20
1110/1110 [==============================] - 2s 2ms/step - loss: 7.8203 - acc: 0.8135
Epoch 11/20
1110/1110 [======

In [26]:
pred = model.predict_classes(inputs[:10])

In [27]:
pred[1]

array([3575, 3575, 3575, 3575, 3575, 3575, 3575, 3575, 3575, 3575, 3575,
       3575, 3575, 3575, 3575])

In [28]:
ctable.decode(pred[0],calc_argmax=False)

[3575 3575 3575 3575 3575 3575 3575 3575 3575 3575 3575 3575 3575 3575
 3575]


'[CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5]=[CH:4][C:3]([C:10]2[CH:15]=[CH:14][C:13]([I:26])=[C:12]([CH3:17])[CH:11]=2)=[C:2]([F:1])[CH:7]=1 [CH3:9][O:8][C:6]1[CH:5

In [29]:
print('Build model...')
from keras.models import Sequential
from keras import layers
RNN = layers.LSTM
HIDDEN_SIZE = 15
BATCH_SIZE = 128
LAYERS = 1
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(3576, input_shape=(15,3576),activation='softmax',name='lstm',return_sequences=True))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#model.add(layers.RepeatVector(15))
# The decoder RNN could be multiple layers stacked or a single layer

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.

#model.add(layers.Dense(len(word_index), activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 15, 3576)          102316512 
Total params: 102,316,512
Trainable params: 102,316,512
Non-trainable params: 0
_________________________________________________________________
